In [44]:
import requests
import pandas as pd
import numpy as np
#import eurostat
import json
import os
import datetime as dt
import openpyxl

In [45]:
import os
print("Working directory:", os.getcwd())

Working directory: /Users/lorenz/Projects/GaR/src


In [74]:
os.chdir('/Users/lorenz/Projects/GaR/')
path = 'src/'
data = 'data/'

In [75]:
# IMPORT INDUSTRIAL PRODUCTION (IP) DATA 

ip = pd.read_csv(os.path.join(path,"ip_by_sector.csv"))

ip = ip[['nace_r2', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'unit']]

ip['Time'] = ip['TIME_PERIOD']

#ip['Time'] = pd.to_datetime(ip['Time'], format="%Y-%m")    

#ip['Time'] = ip['Time'].dt.strftime('%Y/%m')

ip.set_index('Time', inplace=True)

In [48]:
# IMPORT GAS PRICES DATA 

gasp = pd.read_csv("gas_data.csv")

gasp[['Prices']] = gasp[['LSG Natural Gas TTF NL 1st Fut. Day - SETT. PRICE']]

gasp = gasp[['Time','Prices']]

gasp['Time'] = pd.to_datetime(gasp['Time'], format="%m/%d/%y")

month_gasp = gasp.groupby(gasp['Time'].dt.to_period('M')).mean() # group data and make monthly average to match IP data

month_gasp = month_gasp.drop(columns=['Time'])

month_gasp.loc['2004-01':'2023-12']

,Prices
Time,
2004-01,11.360000
2004-02,10.797500
2004-03,10.900435
2004-04,10.974545
2004-05,11.221905
...,...
2023-08,33.523043
2023-09,36.527619
2023-10,43.264545


In [49]:
df = ip.drop(columns=['TIME_PERIOD'])

#df = ip.drop(columns=['TIME_PERIOD']).copy()
#df.index = pd.to_datetime(df.index)
#df.reset_index(inplace=True)

df['geo'] = df['geo'].astype('category')

df['nace_r2'] = df['nace_r2'].astype('category')

df['unit'] = df['unit'].astype('category')

df_ue = df[df['geo'] == 'European Union - 27 countries (from 2020)']

df_ue_c = df_ue[df_ue['nace_r2'] == 'Manufacturing']

df_final = df_ue_c[df_ue_c['unit'] == 'Index, 2021=100']

#df_final = df_final[(df_final['Time'] >= "2004-01") & (df_final['Date'] <= "2023-12")]


df_final = df_final.loc['2004-01':'2023-12']
df_final['GAS_PRICE'] = month_gasp['Prices']

In [51]:
# Load oil supply news shocks from the 'Monthly' sheet
oil_df = pd.read_excel("oilSupplyNewsShocks_2024M06.xlsx", sheet_name="Monthly")

# Convert 'Date' from 'YYYYMM' string to datetime
#oil_df['Date'] = pd.to_datetime(oil_df['Date'], format="%YM%m")
oil_df['Date'] = pd.to_datetime(oil_df['Date'].str.replace("M", ""), format="%Y%m")

# Filter between Jan 2004 and Dec 2023
oil_df = oil_df[(oil_df['Date'] >= "2004-01-01") & (oil_df['Date'] <= "2023-12-31")]

# Rename for merge compatibility
oil_df = oil_df.rename(columns={"Date": "Time"})

In [52]:
oil_df = oil_df.groupby(oil_df['Time'].dt.to_period('M')).mean()

In [53]:
cols = [0]
oil_fixed = oil_df.drop(oil_df.columns[cols], axis=1)
oil_fixed = oil_fixed.rename(columns={"Oil supply surprise series": "OilSurprise"})
oil_fixed = oil_fixed.rename(columns={"Oil supply news shock": "OilNewsShock"})
#oil_fixed[['OilSurprise']] = oil_df[['Oil supply surprise series']]
#oil_fixed[['OilNewsShock']] = oil_df[['Oil supply news shock']]
oil_fixed

,OilSurprise,OilNewsShock
Time,,
2004-01,0.000000,0.441540
2004-02,2.694077,-0.085811
2004-03,-0.667869,0.281320
2004-04,0.000000,-0.603386
2004-05,0.000000,0.596978
...,...,...
2023-08,0.000000,0.263434
2023-09,0.000000,0.470927
2023-10,0.000000,-0.165344


In [54]:
df_final['OilNewsShock'] = oil_fixed['OilNewsShock']
df_final['OilSurprise'] = oil_fixed['OilSurprise']

In [ ]:
# Step 1: Load the Brent Excel
brent_df = pd.read_excel("brenteu.xlsx", sheet_name="Foglio1")

# Step 2: Prepare Brent data
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
brent_df = brent_df.rename(columns={'Europe Brent Spot FOB U$/BBL Daily': 'BRENT_PRICE'})

# Step 3: Monthly average
brent_monthly = brent_df.groupby(brent_df['Date'].dt.to_period('M')).mean(numeric_only=True)

#brent_monthly.index = brent_monthly.index.to_timestamp()
brent_monthly = brent_monthly.loc['2004-01':'2023-12']
brent_monthly = brent_monthly.rename(columns={'Date': 'Time'})
brent_monthly.index = brent_monthly.index.astype(str)

In [84]:
brent_only = brent_monthly[['BRENT_PRICE']]
brent_only[['OilNewsShock', 'OilSurprise']] = oil_fixed[['OilNewsShock', 'OilSurprise']]
brent_only.index = brent_only.index + '-01'
brent_only = brent_only.reset_index()
brent_only['BRENT_PRICE_t1'] = brent_only['BRENT_PRICE'].shift(1)
brent_only['BRENT_PRICE_t1'] = brent_only['BRENT_PRICE_t1'].interpolate(method="nearest").ffill().bfill()

brent_only.to_csv("/Users/lorenz/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/PhD/Progetto/matlab/codice_ipgas/oil_only.csv")

In [ ]:
# Step 4: Merge with your dataset (e.g., df_merged)
df_final['BRENT_PRICE'] = brent_monthly['BRENT_PRICE']

,nace_r2,geo,OBS_VALUE,unit,GAS_PRICE,OilNewsShock,OilSurprise,BRENT_PRICE
Time,,,,,,,,
2004-01,Manufacturing,European Union - 27 countries (from 2020),84.4,"Index, 2021=100",11.360000,0.441540,0.000000,31.236364
2004-02,Manufacturing,European Union - 27 countries (from 2020),85.4,"Index, 2021=100",10.797500,-0.085811,2.694077,30.858500
2004-03,Manufacturing,European Union - 27 countries (from 2020),85.1,"Index, 2021=100",10.900435,0.281320,-0.667869,33.634348
2004-04,Manufacturing,European Union - 27 countries (from 2020),85.9,"Index, 2021=100",10.974545,-0.603386,0.000000,33.590909
2004-05,Manufacturing,European Union - 27 countries (from 2020),86.0,"Index, 2021=100",11.221905,0.596978,0.000000,37.567619
...,...,...,...,...,...,...,...,...
2023-08,Manufacturing,European Union - 27 countries (from 2020),100.8,"Index, 2021=100",33.523043,0.263434,0.000000,86.116087
2023-09,Manufacturing,European Union - 27 countries (from 2020),100.7,"Index, 2021=100",36.527619,0.470927,0.000000,93.722381
2023-10,Manufacturing,European Union - 27 countries (from 2020),100.1,"Index, 2021=100",43.264545,-0.165344,0.000000,90.595909


In [ ]:
#df_ue_c21.to_csv(os.path.join(path, "sample_df.csv"))
df_final.index = df_final.index + '-01'

,nace_r2,geo,OBS_VALUE,unit,GAS_PRICE,OilNewsShock,OilSurprise,BRENT_PRICE
Time,,,,,,,,
2004-01-01,Manufacturing,European Union - 27 countries (from 2020),84.4,"Index, 2021=100",11.360000,0.441540,0.000000,31.236364
2004-02-01,Manufacturing,European Union - 27 countries (from 2020),85.4,"Index, 2021=100",10.797500,-0.085811,2.694077,30.858500
2004-03-01,Manufacturing,European Union - 27 countries (from 2020),85.1,"Index, 2021=100",10.900435,0.281320,-0.667869,33.634348
2004-04-01,Manufacturing,European Union - 27 countries (from 2020),85.9,"Index, 2021=100",10.974545,-0.603386,0.000000,33.590909
2004-05-01,Manufacturing,European Union - 27 countries (from 2020),86.0,"Index, 2021=100",11.221905,0.596978,0.000000,37.567619
...,...,...,...,...,...,...,...,...
2023-08-01,Manufacturing,European Union - 27 countries (from 2020),100.8,"Index, 2021=100",33.523043,0.263434,0.000000,86.116087
2023-09-01,Manufacturing,European Union - 27 countries (from 2020),100.7,"Index, 2021=100",36.527619,0.470927,0.000000,93.722381
2023-10-01,Manufacturing,European Union - 27 countries (from 2020),100.1,"Index, 2021=100",43.264545,-0.165344,0.000000,90.595909


In [70]:
df_final = df_final.rename(columns={"OBS_VALUE": "IP"})

In [76]:
df_final.to_csv(os.path.join(data,"df_final.csv"))